## This notebook demonstrates the use of the adapted Adversarial Debiasing algorithm.

The code has been updated to Tensorflow 2.X and can now be used to learn a fair classifier with respect to multiple sensitive attributes simultaneously.

References:

[Mitigating UnwantedBiases with Adversarial Learning](https://arxiv.org/pdf/1801.07593.pdf)  
  
B. H. Zhang, B. Lemoine, and M. Mitchell,    
AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.

In [1]:

import sys
sys.path.append("../")

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Markdown, display
from sklearn.metrics import roc_auc_score

# import adapted AdversarialDebiasing code
from aif360.algorithms.inprocessing.multi_attribute_adversarial_debiasing import *



`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

### Helper functions

In [2]:
def accuracy(predicted_labels, true):
    return tf.reduce_mean(tf.cast(tf.equal(predicted_labels, true), dtype=tf.float32))

### 1 . Load dataset

In [3]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_compas()#load_preproc_data_adult()#

privileged_groups = [{'sex': 1}, {'race': 1}]
unprivileged_groups = [{'sex': 0}, {'race': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [4]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(3694, 10)


#### Favorable and unfavorable labels

0.0 1.0


#### Protected attribute names

['sex', 'race']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['sex', 'race', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'priors_count=0', 'priors_count=1 to 3', 'priors_count=More than 3', 'c_charge_degree=F', 'c_charge_degree=M']


#### Evaluate metric for original training data

Race and sex are the two sensitive attributes

In [5]:
for unprivileged_group, privileged_group in zip (unprivileged_groups, privileged_groups):
    print('Sensitive attribute : ', list(unprivileged_group.keys())[0], '\n')


    # Metric for the original dataset
    metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                                unprivileged_groups=[unprivileged_group],
                                                privileged_groups=[privileged_group])
    
    print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
    metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                                unprivileged_groups=[unprivileged_group],
                                                privileged_groups=[privileged_group])
    print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())
    print('\n'*2)

Sensitive attribute :  sex 

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.122968
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.162710



Sensitive attribute :  race 

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.136772
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.121692





#### Scale features

In [6]:
min_max_scaler = MinMaxScaler()#MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)



### 2 . Debias multiple sensitive attributes 

3 performance metrics and 2 Fairness metrics to evaluate the implementation. 
> <b> Performance metrics:  </b>  
> - Accuracy  
> - Balanced Accuracy  
> - AUC score  

> <b> Fairness metrics:  </b>   
> - Equal opportunity difference  
> - Average odds Difference  

##### NOTE:
Current implementation supports an adversary that tries to enforce equality of Odds  
since it gets both yHat and y during training (see [Mitigating UnwantedBiases with Adversarial Learning](https://arxiv.org/pdf/1801.07593.pdf)  )

In [7]:
# Example dataset
features = dataset_orig_train.features
labels = dataset_orig_train.labels
protected_attributes = dataset_orig_train.protected_attributes

dataset_orig_train.protected_attribute_names

['sex', 'race']

#### Training the model

The model consists of a classifier and an adversary network. At each training iteration, the classifier learns to predict a target label, and the adversary takes the classifier's predictions as well as  
the true target label, and learns to predict multiple sensitive attribute values (e.g. race , sex) for that instance. Each model can be pretrained for several epochs before initiating adversarial learning.  
The adversary can be provided with weights for each sensitive attribute, which is used in the loss computation.

In [8]:
# Define training parameters
nb_pretrain = 10
batch_size = 64
total_epochs = 40

# weights for 'sex', 'race'
loss_weights  = [3, 5]

In [9]:
# Create a trainer
trainer = AdversarialDebiasor(loss_weights = loss_weights)

# Pretrain the classifier model
trainer.pretrain_classifier(features, labels, num_epochs=nb_pretrain, batch_size=batch_size)

Pretraining Classifier - Epoch 1, Loss: 0.5662161707878113
Pretraining Classifier - Epoch 2, Loss: 0.5489364862442017
Pretraining Classifier - Epoch 3, Loss: 0.6741243004798889
Pretraining Classifier - Epoch 4, Loss: 0.5755788683891296
Pretraining Classifier - Epoch 5, Loss: 0.5527998208999634
Pretraining Classifier - Epoch 6, Loss: 0.6184147000312805
Pretraining Classifier - Epoch 7, Loss: 0.6741035580635071
Pretraining Classifier - Epoch 8, Loss: 0.549817681312561
Pretraining Classifier - Epoch 9, Loss: 0.5531249642372131
Pretraining Classifier - Epoch 10, Loss: 0.7086078524589539


#### Evaluate test set classification metrics after pre-training classifier

In [10]:
# Evaluate metrics

metrics = trainer.get_classification_metrics(dataset_orig_test)

for key, classified_metric_debiasing_test in metrics.items():
    print('Sensitive attribute : ', key)

    eod = classified_metric_debiasing_test.equal_opportunity_difference()
    aod = classified_metric_debiasing_test.average_odds_difference()
    ti = classified_metric_debiasing_test.theil_index()
    print('\tEqual opportunity diff. :', eod)
    print('\tAv. Odds diff.          :', aod, '\n'*2)


predicted_labels = trainer.predict(dataset_orig_test.features)
true_labels = dataset_orig_test.labels

TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)

# Should not change since only trained adversary
print('\nAccuracy        : ', accuracy(predicted_labels, true_labels).numpy())
print('Balanced accuracy : ', bal_acc_debiasing_test)
print('AUC score         : ', roc_auc_score(true_labels, trainer.predict_proba(dataset_orig_test.features)))


Sensitive attribute :  sex
	Equal opportunity diff. : -0.19152323847396535
	Av. Odds diff.          : -0.26593566325003604 


Sensitive attribute :  race
	Equal opportunity diff. : -0.1951045318789767
	Av. Odds diff.          : -0.22362652902240815 



Accuracy        :  0.6445707
Balanced accuracy :  0.639877867033843
AUC score         :  0.6988965543680187


#### Pretrain the adversary on trained classifier's outputs

In [11]:

trainer.pretrain_adversary(features, labels, protected_attributes, num_epochs=nb_pretrain, batch_size=batch_size)

Pretraining Adversary - Epoch 1, Loss: 4.9558305740356445
Pretraining Adversary - Epoch 2, Loss: 4.603690147399902
Pretraining Adversary - Epoch 3, Loss: 4.303704261779785
Pretraining Adversary - Epoch 4, Loss: 5.025561332702637
Pretraining Adversary - Epoch 5, Loss: 4.46227502822876
Pretraining Adversary - Epoch 6, Loss: 4.4050188064575195
Pretraining Adversary - Epoch 7, Loss: 4.613255977630615
Pretraining Adversary - Epoch 8, Loss: 4.509513854980469
Pretraining Adversary - Epoch 9, Loss: 4.4759440422058105
Pretraining Adversary - Epoch 10, Loss: 4.689357280731201


#### Train both models together

In [12]:
# Train both models together
trainer.train(features, labels, protected_attributes, num_epochs=total_epochs, batch_size=batch_size)

predicted_labels = trainer.predict(dataset_orig_test.features)

Epoch 1, Classifier Loss: 1.0748538970947266, Adversary Loss: 6.006758213043213
Epoch 2, Classifier Loss: 0.7039031386375427, Adversary Loss: 4.691508769989014
Epoch 3, Classifier Loss: 0.6419263482093811, Adversary Loss: 4.874557018280029
Epoch 4, Classifier Loss: 0.6589154601097107, Adversary Loss: 4.840798377990723
Epoch 5, Classifier Loss: 0.6023615002632141, Adversary Loss: 4.959635257720947
Epoch 6, Classifier Loss: 0.6404706835746765, Adversary Loss: 4.265321731567383
Epoch 7, Classifier Loss: 0.6090013384819031, Adversary Loss: 5.116223335266113
Epoch 8, Classifier Loss: 0.6836777925491333, Adversary Loss: 4.832919120788574
Epoch 9, Classifier Loss: 0.6418229937553406, Adversary Loss: 4.165708541870117
Epoch 10, Classifier Loss: 0.6556251049041748, Adversary Loss: 4.947446823120117
Epoch 11, Classifier Loss: 0.5766992568969727, Adversary Loss: 4.835245132446289
Epoch 12, Classifier Loss: 0.6414293050765991, Adversary Loss: 5.631744384765625
Epoch 13, Classifier Loss: 0.64227461

#### Evaluate test set classification metrics after debiasing

In [13]:
metrics = trainer.get_classification_metrics(dataset_orig_test)

for key, classified_metric_debiasing_test in metrics.items():
    print('Sensitive attribute : ', key, '\n')

    TPR = classified_metric_debiasing_test.true_positive_rate()
    TNR = classified_metric_debiasing_test.true_negative_rate()
    bal_acc_debiasing_test = 0.5*(TPR+TNR)
    eod = classified_metric_debiasing_test.equal_opportunity_difference()
    aod = classified_metric_debiasing_test.average_odds_difference()
    ti = classified_metric_debiasing_test.theil_index()
    print('\tEqual opportunity diff. :', eod)
    print('\tAv. Odds diff.          :', aod, '\n'*2)
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)



print('Accuracy          : ', accuracy(predicted_labels, true_labels).numpy())
print('Balanced accuracy : ', bal_acc_debiasing_test)
print('AUC score         : ', roc_auc_score(true_labels, trainer.predict_proba(dataset_orig_test.features)))


Sensitive attribute :  sex 

	Equal opportunity diff. : -0.06782807257363
	Av. Odds diff.          : -0.056150669204562464 


Sensitive attribute :  race 

	Equal opportunity diff. : -0.042509237289136026
	Av. Odds diff.          : -0.03371920122780872 


Accuracy          :  0.6433081
Balanced accuracy :  0.6382773277703147
AUC score         :  0.6775512388205847


In [14]:
dmetrics = trainer.get_dataset_metrics(dataset_orig_test)

for key, classified_metric_debiasing_test in metrics.items():
    print('Sensitive attribute : ', key)
    print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % classified_metric_debiasing_test.mean_difference())
    print('\n')

Sensitive attribute :  sex
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.102985


Sensitive attribute :  race
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.067732


